<a href="https://colab.research.google.com/github/Eric-Manzi/UNETVariantsModels/blob/main/ASPP_SE_UNET_Dense_skincancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [ ]:
!pip install -q pydot
!pip install -q graphviz
!apt install libgraphviz-dev

# Restart runtime after installation
import os
os.kill(os.getpid(), 9)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libgvc6-plugins-gtk
  librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
0 upgraded, 9 newly installed, 0 to remove and 54 not upgraded.
Need to get 2,434 kB of archives.
After this operation, 7,681 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgtk2.0-common all 2.24.33-2ubuntu2.1 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgtk2.0-0 amd64 2.24.33-2ubuntu2.1 [2,038 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgail18 amd64 2.24.33-2ubuntu2.1 [15.9 kB]
Get:4 http://archive.ubun

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
import zipfile
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, concatenate, Activation, BatchNormalization
from tensorflow.keras.layers import UpSampling2D, GlobalAveragePooling2D, Dense, Multiply, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ConvLSTM2D, Add, Bidirectional, Dropout, Concatenate
from tensorflow.keras.layers import Reshape
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Unzip the dataset
zip_path = '/content/drive/MyDrive/Datasets/ham1000-segmentation-and-classification.zip'
extract_path = '/content/ham1000_dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)


Dataset extracted to: /content/ham1000_dataset


In [ ]:
# Assuming the images and masks are stored in separate folders named 'images' and 'masks' within the extracted directory
extract_path = '/content/ham1000_dataset'
image_folder = os.path.join(extract_path, 'images')
mask_folder = os.path.join(extract_path, 'masks')

# Define allowed image and mask extensions
image_extensions = ('.jpg', '.jpeg')  # For image files
mask_extensions = ('.png',)  # For mask files

# Get the list of files in the image and mask folders, filtering by extension
image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith(image_extensions)])
mask_files = sorted([f for f in os.listdir(mask_folder) if f.lower().endswith(mask_extensions)])

# Print the number of images and masks for debugging
print("Number of image files found:", len(image_files))
print("Number of mask files found:", len(mask_files))

# Print the first few filenames for debugging
print("First 5 image filenames:", image_files[:5])
print("First 5 mask filenames:", mask_files[:5])

# Ensure the number of images and masks match
# Check if the number of images and masks are equal. If not, print detailed information
# and raise the assertion error to halt execution.
if len(image_files) != len(mask_files):
    print(f"Number of image files: {len(image_files)}")
    print(f"Number of mask files: {len(mask_files)}")
    print("Image files:", image_files)  # Print all image filenames
    print("Mask files:", mask_files)  # Print all mask filenames
    raise AssertionError("The number of images and masks must be equal.")
#Original Assertion
#assert len(image_files) == len(mask_files), "The number of images and masks must be equal."


# Shuffle the dataset
image_files, mask_files = shuffle(image_files, mask_files, random_state=42)

# Split the dataset: 2500 for training, 500 for validation, 300 for testing
train_image_files = image_files[:2500]
train_mask_files = mask_files[:2500]

val_image_files = image_files[2500:3000]
val_mask_files = mask_files[2500:3000]

test_image_files = image_files[3000:3300]
test_mask_files = mask_files[3000:3300]

# Now you have:
# 2500 training images/masks
# 500 validation images/masks
# 300 testing images/masks


# Verify if the folders exist
print("Images folder:", image_folder)
print("Masks folder:", mask_folder)
print("Number of images:", len(os.listdir(image_folder)))
print("Number of masks:", len(os.listdir(mask_folder)))


Number of image files found: 10015
Number of mask files found: 10015
First 5 image filenames: ['ISIC_0024306.jpg', 'ISIC_0024307.jpg', 'ISIC_0024308.jpg', 'ISIC_0024309.jpg', 'ISIC_0024310.jpg']
First 5 mask filenames: ['ISIC_0024306_segmentation.png', 'ISIC_0024307_segmentation.png', 'ISIC_0024308_segmentation.png', 'ISIC_0024309_segmentation.png', 'ISIC_0024310_segmentation.png']
Images folder: /content/ham1000_dataset/images
Masks folder: /content/ham1000_dataset/masks
Number of images: 10017
Number of masks: 10015


In [ ]:
class DataGenerator(Sequence):
    def __init__(self, image_folder, mask_folder, image_files, mask_files, batch_size, img_size, shuffle=True, augment=True):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.image_files = image_files
        self.mask_files = mask_files
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.augment = augment  # Flag to control augmentation
        self.indices = np.arange(len(self.image_files))
        self.on_epoch_end()

        # Define data augmentation parameters
        self.datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

    def __len__(self):
        return len(self.image_files) // self.batch_size

    def __getitem__(self, index):
        start_index = index * self.batch_size
        end_index = min((index + 1) * self.batch_size, len(self.image_files))
        batch_indices = self.indices[start_index:end_index]

        images = []
        masks = []

        for i in batch_indices:
            image_path = os.path.join(self.image_folder, self.image_files[i])
            mask_path = os.path.join(self.mask_folder, self.mask_files[i])

            image = cv2.imread(image_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            image = cv2.resize(image, self.img_size)
            mask = cv2.resize(mask, self.img_size)

            # Apply augmentation if enabled
            if self.augment:
                # Combine image and mask for simultaneous transformation
                seed = np.random.randint(0, 2**32 - 1)  # Ensure consistent transformations
                image = self.datagen.random_transform(image, seed=seed)
                mask = self.datagen.random_transform(mask[..., np.newaxis], seed=seed)[..., 0]  # Augment mask separately

            # Cast data types and normalize
            image = tf.cast(image, tf.float32) / 255.0
            mask = tf.cast(mask, tf.float32) / 255.0

            images.append(image)
            masks.append(mask)

        return np.array(images), np.array(masks)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
# Define the batch size and image size
batch_size = 8
img_size = (256, 256)

# Create DataGenerators for each split
# Include image_files and mask_files in the DataGenerator instantiation
train_generator = DataGenerator(
    image_folder=image_folder,
    mask_folder=mask_folder,
    image_files=train_image_files,  # Pass train_image_files here
    mask_files=train_mask_files,  # Pass train_mask_files here
    batch_size=batch_size,
    img_size=img_size,
    shuffle=True
)

val_generator = DataGenerator(
    image_folder=image_folder,
    mask_folder=mask_folder,
    image_files=val_image_files,  # Pass val_image_files here
    mask_files=val_mask_files,  # Pass val_mask_files here
    batch_size=batch_size,
    img_size=img_size,
    shuffle=False
)

test_generator = DataGenerator(
    image_folder=image_folder,
    mask_folder=mask_folder,
    image_files=test_image_files,  # Pass test_image_files here
    mask_files=test_mask_files,  # Pass test_mask_files here
    batch_size=batch_size,
    img_size=img_size,
    shuffle=False
)

NameError: name 'image_folder' is not defined

In [ ]:
def dice_score(y_true, y_pred):
    #Calculates the Dice score.
    y_true = tf.cast(y_true, tf.float32)  # Cast y_true to float32
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def iou_score(y_true, y_pred):
    #Calculates the Intersection over Union (IoU) score.
    y_true = tf.cast(y_true, tf.float32)  # Cast y_true to float32
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + 1) / (union + 1)

In [ ]:
def pixel_accuracy(y_true, y_pred):
    # Calculates Pixel Accuracy.
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    correct_pixels = K.sum(K.equal(y_true, y_pred))
    total_pixels = K.sum(K.cast(K.not_equal(y_true, -1), tf.float32))  # Exclude background pixels (if -1 is used)
    return correct_pixels / (total_pixels + K.epsilon())

def precision(y_true, y_pred):
    # Calculates Precision.
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(y_true * y_pred)
    false_positives = K.sum((1 - y_true) * y_pred)
    return true_positives / (true_positives + false_positives + K.epsilon())

def recall(y_true, y_pred):
    # Calculates Recall.
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(y_true * y_pred)
    false_negatives = K.sum(y_true * (1 - y_pred))
    return true_positives / (true_positives + false_negatives + K.epsilon())

In [ ]:
def dice_loss(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


In [ ]:
# Defining the Multiscale Attention Unet model with ASPP and SE

# Squeeze-and-Excitation Block
def se_block(input_tensor, ratio=16):
    filters = input_tensor.shape[-1]
    se_shape = (1, 1, filters)

    # Squeeze: Global Average Pooling
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)

    # Excitation: Scale the input tensor
    se = Reshape((1, 1, filters))(se)
    x = Multiply()([input_tensor, se])
    return x

# Atrous Spatial Pyramid Pooling (ASPP) Block
def aspp_block(input_tensor, filters):
    atrous_rates = [1, 6, 12, 18]
    aspp_features = []

    for rate in atrous_rates:
        x = Conv2D(filters, (3, 3), padding='same', dilation_rate=rate)(input_tensor)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        aspp_features.append(x)

    # Concatenate ASPP features
    x = concatenate(aspp_features, axis=-1)
    x = Conv2D(filters, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# U-Net Model with SE and ASPP
def build_model1(input_shape=(256, 256, 3), base_filters=64):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(base_filters, (3, 3), padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    c1 = Conv2D(base_filters, (3, 3), padding='same')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(base_filters * 2, (3, 3), padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    c2 = Conv2D(base_filters * 2, (3, 3), padding='same')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(base_filters * 4, (3, 3), padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    c3 = Conv2D(base_filters * 4, (3, 3), padding='same')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(base_filters * 8, (3, 3), padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    c4 = Conv2D(base_filters * 8, (3, 3), padding='same')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck with ASPP and SE Module
    b1 = aspp_block(p4, base_filters * 16)
    b1 = se_block(b1)

    # Decoder
    u1 = UpSampling2D((2, 2))(b1)
    u1 = concatenate([u1, c4])
    u1 = Conv2D(base_filters * 8, (3, 3), padding='same')(u1)
    u1 = BatchNormalization()(u1)
    u1 = Activation('relu')(u1)
    u1 = Conv2D(base_filters * 8, (3, 3), padding='same')(u1)
    u1 = BatchNormalization()(u1)
    u1 = Activation('relu')(u1)

    u2 = UpSampling2D((2, 2))(u1)
    u2 = concatenate([u2, c3])
    u2 = Conv2D(base_filters * 4, (3, 3), padding='same')(u2)
    u2 = BatchNormalization()(u2)
    u2 = Activation('relu')(u2)
    u2 = Conv2D(base_filters * 4, (3, 3), padding='same')(u2)
    u2 = BatchNormalization()(u2)
    u2 = Activation('relu')(u2)

    u3 = UpSampling2D((2, 2))(u2)
    u3 = concatenate([u3, c2])
    u3 = Conv2D(base_filters * 2, (3, 3), padding='same')(u3)
    u3 = BatchNormalization()(u3)
    u3 = Activation('relu')(u3)
    u3 = Conv2D(base_filters * 2, (3, 3), padding='same')(u3)
    u3 = BatchNormalization()(u3)
    u3 = Activation('relu')(u3)

    u4 = UpSampling2D((2, 2))(u3)
    u4 = concatenate([u4, c1])
    u4 = Conv2D(base_filters, (3, 3), padding='same')(u4)
    u4 = BatchNormalization()(u4)
    u4 = Activation('relu')(u4)
    u4 = Conv2D(base_filters, (3, 3), padding='same')(u4)
    u4 = BatchNormalization()(u4)
    u4 = Activation('relu')(u4)

    # Output Layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(u4)

    model = Model(inputs, outputs)
    return model


In [ ]:
from tensorflow.keras.layers import Input, Activation, GlobalAveragePooling2D, Dense, Multiply, Lambda, Dropout

# Squeeze-and-Excitation Block
def se_block(input_tensor, ratio=16):
    filters = input_tensor.shape[-1]
    se_shape = (1, 1, filters)

    # Squeeze: Global Average Pooling
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)

    # Excitation: Scale the input tensor
    se = Reshape((1, 1, filters))(se)
    x = Multiply()([input_tensor, se])
    return x

# Atrous Spatial Pyramid Pooling (ASPP) Block
def aspp_block(input_tensor, filters):
    atrous_rates = [1, 6, 12, 18]
    aspp_features = []

    for rate in atrous_rates:
        x = Conv2D(filters, (3, 3), padding='same', dilation_rate=rate)(input_tensor)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        aspp_features.append(x)

    # Concatenate ASPP features
    x = concatenate(aspp_features, axis=-1)
    x = Conv2D(filters, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# Dense Block (from DenseNet)
def dense_block(x, num_layers, growth_rate):
    for _ in range(num_layers):
        conv = BatchNormalization()(x)
        conv = Activation('relu')(conv)
        conv = Conv2D(growth_rate, (3, 3), padding='same')(conv)
        x = concatenate([x, conv])  # Concatenate input and output
    return x

# U-Net Model with SE, ASPP, and Dense Blocks
def build_model2(input_shape=(256, 256, 3), base_filters=128, growth_rate=32, dense_layers=4):
    inputs = Input(input_shape)

    # Encoder
    c1 = dense_block(inputs, dense_layers, growth_rate)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = dense_block(p1, dense_layers, growth_rate)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = dense_block(p2, dense_layers, growth_rate)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = dense_block(p3, dense_layers, growth_rate)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck with ASPP and SE Module
    b1 = aspp_block(p4, base_filters * 4)
    b1 = se_block(b1)

    # Decoder
    u1 = UpSampling2D((2, 2))(b1)
    u1 = concatenate([u1, c4])
    u1 = dense_block(u1, dense_layers, growth_rate)

    u2 = UpSampling2D((2, 2))(u1)
    u2 = concatenate([u2, c3])
    u2 = dense_block(u2, dense_layers, growth_rate)

    u3 = UpSampling2D((2, 2))(u2)
    u3 = concatenate([u3, c2])
    u3 = dense_block(u3, dense_layers, growth_rate)

    u4 = UpSampling2D((2, 2))(u3)
    u4 = concatenate([u4, c1])
    u4 = dense_block(u4, dense_layers, growth_rate)

    # Output Layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(u4)

    model = Model(inputs, outputs)
    return model



In [ ]:
# Build the model
model = build_model2()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', dice_score, iou_score, pixel_accuracy, precision, recall])

NameError: name 'build_model2' is not defined

In [ ]:
model.summary()

In [ ]:
# Build the model
model = build_model1()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', dice_score, iou_score])

In [ ]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_44 (Conv2D)        │ (None, 256, 256, 128)  │          3,584 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_42    │ (None, 256, 256, 128)  │            512 │ conv2d_44[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_42             │ (None, 256, 256, 128)  │              0 │ batch_normalization_4… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_45 (Conv2D)        │ (None, 256, 256, 128)  │        147,584 │ activation_42[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_43    │ (None, 256, 256, 128)  │            512 │ conv2d_45[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_43             │ (None, 256, 256, 128)  │              0 │ batch_normalization_4… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_8           │ (None, 128, 128, 128)  │              0 │ activation_43[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_46 (Conv2D)        │ (None, 128, 128, 256)  │        295,168 │ max_pooling2d_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_44    │ (None, 128, 128, 256)  │          1,024 │ conv2d_46[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_44             │ (None, 128, 128, 256)  │              0 │ batch_normalization_4… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_47 (Conv2D)        │ (None, 128, 128, 256)  │        590,080 │ activation_44[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_45    │ (None, 128, 128, 256)  │          1,024 │ conv2d_47[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_45             │ (None, 128, 128, 256)  │              0 │ batch_normalization_4… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_9      

 Total params: 161,756,289 (617.05 MB)

 Trainable params: 161,720,449 (616.91 MB)

 Non-trainable params: 35,840 (140.00 KB)

In [ ]:
#plot_model(model, to_file='model_plot.png', show_shapes=False, show_layer_names=False)

In [ ]:
#model checkpoints
with tf.device('/GPU:0'):
      checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_mau.keras', verbose=1, save_best_only=True)

      callbacks=[ tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
                tf.keras.callbacks.TensorBoard(log_dir='logs')]

In [ ]:
# Train the model
history = model.fit(train_generator, epochs=30, validation_data=val_generator, callbacks=callbacks)




In [ ]:
import matplotlib.pyplot as plt

def plot_training_validation(history):
    # Plot accuracy, Dice Score, and IoU Score
    metrics_to_plot = ['accuracy', 'dice_score', 'iou_score']
    num_metrics = len(metrics_to_plot)

    fig, axes = plt.subplots(1, num_metrics, figsize=(6 * num_metrics, 5))

    for i, metric in enumerate(metrics_to_plot):
        axes[i].plot(history.history[metric], label=f'Training {metric.capitalize()}')
        axes[i].plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')
        axes[i].set_title(f'Training vs Validation {metric.capitalize()}')
        axes[i].set_xlabel('Epochs')
        axes[i].set_ylabel(metric.capitalize())
        axes[i].legend()
        axes[i].grid(True)

    plt.tight_layout()
    plt.show()

    # Plot loss
    plt.figure(figsize=(6, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot precision and recall if available
    if 'precision' in history.history and 'recall' in history.history:
        fig, ax = plt.subplots(1, 2, figsize=(12, 5))

        # Precision
        ax[0].plot(history.history['precision'], label='Training Precision')
        ax[0].plot(history.history['val_precision'], label='Validation Precision')
        ax[0].set_title('Training vs Validation Precision')
        ax[0].set_xlabel('Epochs')
        ax[0].set_ylabel('Precision')
        ax[0].legend()
        ax[0].grid(True)

        # Recall
        ax[1].plot(history.history['recall'], label='Training Recall')
        ax[1].plot(history.history['val_recall'], label='Validation Recall')
        ax[1].set_title('Training vs Validation Recall')
        ax[1].set_xlabel('Epochs')
        ax[1].set_ylabel('Recall')
        ax[1].legend()
        ax[1].grid(True)

        plt.tight_layout()
        plt.show()

# Use the function with your history object after training
plot_training_validation(history)
```

""" Key Points:
1. **Metrics Included**: This code plots `accuracy`, `dice_score`, `iou_score`, `precision`, and `recall` metrics. Add more metrics if needed.
2. **Loss Plot**: A separate plot for loss is included.
3. **Grid Layout**: The code dynamically adjusts for the number of metrics to ensure clean visualization.

Replace `history` with the actual training history returned by the `model.fit` function. This way, you can comprehensively evaluate the training and validation performance of your model."""

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy, test_dice, test_iou = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Dice Score: {test_dice}")
print(f"Test IoU Score: {test_iou}")


In [ ]:
# Load the TensorBoard extension
%load_ext tensorboard
# Start TensorBoard and specify the log directory
%tensorboard --logdir logs/fit

!pip install -q pyngrok
from pyngrok import ngrok
# Start TensorBoard as a background process
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
%tensorboard --logdir logs/fit --port=6006 &

# Start ngrok to tunnel the port
public_url = ngrok.connect(port="6006")
print(f"TensorBoard URL: {public_url}")
#The code above will output a URL. Open this URL in your local browser to view the TensorBoard metrics running in Colab.

In [ ]:
# Select a single image from your dataset for demonstration
sample_image = train_image_files[0]  # Replace with any image in your dataset

# Reshape the image to add a batch dimension (required for ImageDataGenerator)
sample_image = np.expand_dims(sample_image, 0)

# Create a generator to produce augmented versions of this image
augmented_images = datagen.flow(sample_image, batch_size=1)

# Plot a few augmented images
plt.figure(figsize=(10, 10))
for i in range(9):  # Displaying 9 augmented samples
    aug_img = next(augmented_images)[0].astype('uint8')  # Get the next augmented image
    plt.subplot(3, 3, i + 1)
    plt.imshow(aug_img)
    plt.axis('off')
plt.suptitle('Augmented Images')
plt.show()


ValueError: could not convert string to float: 'ISIC_0025923.jpg'